In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

sns.set_style('whitegrid')
sns.set_palette('pastel', 8)

In [2]:
!pwd

/home/ec2-user/SageMaker/tam_topic_modelling


In [3]:
file_path = 'data/EC2_India_Cases_WithTS-Latest.xlsx'
data = pd.read_excel(file_path, engine='openpyxl', skiprows=2)
data.head(5)

,case_id,customer_name,service,case_billing_region,customer_billing_country_name,comm_owner_agent_login,comm_body,case_creation_cal_date,comm_subject,comm_date_utc,case_severity
0,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,rayaans,"Hi Team, we have instance currently undergoin...","Sep 14, 2020",Chat: high cpu utilization of server,"September 14, 2020 6:40:59",4
1,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,NaN,"Hi Team, we have instance currently undergoin...","Sep 14, 2020",Chat: high cpu utilization of server,"September 14, 2020 7:03:26",4
2,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,rayaans,"{""transcript"":[{""action"":""TRANSCRIPT_START"",""d...","Sep 14, 2020",NaN,"September 14, 2020 7:03:36",4
3,7380321081,Edelweiss Financial Services Ltd,Elastic Compute Cloud (EC2 - Windows),APAC,INDIA,shrmjj,"Hi team, we are not able to fetch ec2 system ...","Sep 14, 2020",Call: unable to fetch system logs,"September 14, 2020 7:11:41",4
4,7380321081,Edelweiss Financial Services Ltd,Elastic Compute Cloud (EC2 - Windows),APAC,INDIA,shrmjj,Phone call: you and Amazon,"Sep 14, 2020",Inbound Call,"September 14, 2020 7:14:29",4


In [4]:
data.sort_values(by=['case_id', 'comm_date_utc'], inplace=True)
data.reset_index(inplace=True)
data.drop(columns=['index'], inplace=True)
data.head(10)

,case_id,customer_name,service,case_billing_region,customer_billing_country_name,comm_owner_agent_login,comm_body,case_creation_cal_date,comm_subject,comm_date_utc,case_severity
0,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,rayaans,"Hi Team, we have instance currently undergoin...","Sep 14, 2020",Chat: high cpu utilization of server,"September 14, 2020 6:40:59",4
1,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,NaN,"Hi Team, we have instance currently undergoin...","Sep 14, 2020",Chat: high cpu utilization of server,"September 14, 2020 7:03:26",4
2,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,rayaans,"{""transcript"":[{""action"":""TRANSCRIPT_START"",""d...","Sep 14, 2020",NaN,"September 14, 2020 7:03:36",4
3,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,NaN,"Hi John, Sorry, we could not complete our dis...","Sep 14, 2020",Chat: high cpu utilization of server,"September 14, 2020 7:48:52",4
4,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,rayaans,"{""transcript"":[{""action"":""TRANSCRIPT_START"",""d...","Sep 14, 2020",NaN,"September 14, 2020 7:49:28",4
5,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,rayaans,"Hello Shruti, Thank you for your time and pat...","Sep 14, 2020",RE:[CASE 7380196011] Chat: high cpu utilizatio...,"September 14, 2020 9:42:41",4
6,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,arizona,"Hello, We haven't heard back from you regardi...","Sep 14, 2020",Chat: high cpu utilization of server,"September 21, 2020 10:19:39",4
7,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,arizona,Please let us know if we helped resolve your i...,"Sep 14, 2020",Chat: high cpu utilization of server,"September 24, 2020 12:13:02",4
8,7380321081,Edelweiss Financial Services Ltd,Elastic Compute Cloud (EC2 - Windows),APAC,INDIA,shrmjj,"Hi team, we are not able to fetch ec2 system ...","Sep 14, 2020",Call: unable to fetch system logs,"September 14, 2020 7:11:41",4
9,7380321081,Edelweiss Financial Services Ltd,Elastic Compute Cloud (EC2 - Windows),APAC,INDIA,shrmjj,Phone call: you and Amazon,"Sep 14, 2020",Inbound Call,"September 14, 2020 7:14:29",4


In [5]:
first_response_index = data['case_id'].drop_duplicates(keep='first').index.values
data = data[data.index.isin(first_response_index)]
data.shape

(2608, 11)

In [6]:
data.to_csv('data/ntm_first_response_data.csv', index=False)

In [7]:
data = pd.read_csv('data/ntm_first_response_data.csv')
data.head()

,case_id,customer_name,service,case_billing_region,customer_billing_country_name,comm_owner_agent_login,comm_body,case_creation_cal_date,comm_subject,comm_date_utc,case_severity
0,7380196011,RBL Bank Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,rayaans,"Hi Team, we have instance currently undergoin...","Sep 14, 2020",Chat: high cpu utilization of server,"September 14, 2020 6:40:59",4
1,7380321081,Edelweiss Financial Services Ltd,Elastic Compute Cloud (EC2 - Windows),APAC,INDIA,shrmjj,"Hi team, we are not able to fetch ec2 system ...","Sep 14, 2020",Call: unable to fetch system logs,"September 14, 2020 7:11:41",4
2,7380672281,Go Digit General Insurance Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,siljeurc,"Hi Raghavendra, Just a quick recap of our ch...","Sep 14, 2020",RE:[CASE 7380672281] no command is not exeucting,"September 14, 2020 10:48:40",4
3,7380713151,Go Digit General Insurance Limited,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,lonesaji,"Hello Balaji, This is Sajid from AWS Premium ...","Sep 14, 2020",RE:[CASE 7380713151] Chat: instance went to 1/2,"September 14, 2020 12:57:32",1
4,7380937061,ALL_DEPRECATED,Elastic Compute Cloud (EC2 - Linux),APAC,INDIA,pauyen,"实例已经终止了,能恢复吗,数据能恢复吗 Instance ID(s): i-08fdcb5...","Sep 14, 2020","实例已经终止了,能恢复吗","September 14, 2020 11:54:03",3


In [8]:
!pip install textblob
!pip install spacy-langdetect
!pip install language-detector
!pip install symspellpy
!pip install sentence-transformers

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [9]:
!pip install gensim
!pip install stop-words

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [10]:
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
from nltk.corpus import wordnet
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords 

In [11]:
import os
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import datetime
import time
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import nltk

In [12]:
abbr_3 = [
    'ec2', 'ami', 'ebs', 'kms', 'cmk', 'url', 'web', 'oem', 'asn', 'gps', 'jre', 'isn', 'jmx', 'gdb', 'eol', 'gpt', 'mbr', 
    'isp', 'ide', 'phd', 'faq', 'gcc', 'vnc', 'udp', 'ntp', 'nfs', 'gpu', 'pid', 'lvm', 'gui', 'ecs', 'pem', 'awk', 'ecs', 'dms',
    'msg', 'pfa', 'rdp', 'pci', 'sep', 'fri', 'tue', 'feb', 'thu', 'gpl', 'sns', 'sme', 'acl', 'cmd', 'ecr', 'aug', 'mon', 'vpn', 
    'ssl', 'npm', 'vpn', 'rds', 'efs', 'dns', 'tcp', 'sql', 'ena', 'dec', 'emr', 'gmt', 'vpc', 'nat', 'acm', 'arn', 'cli', 'sdk',
    'api', 'cpu'
]

abbr_5 = ['mysql', 'testp', 'repos','posix','chmod','stdev','geoip','nslcd', 'upgra', 'nacls', 'utils','cyber','iface','glibc','inode']

abbr_4 = ['etho',
'nacl',
'inot',
'wmic',
'ntfs',
'arti', 
'vpcs',
'usef',
'usin',
'trai'
'http',
'repo',
'dhcp',
'smtp',
'rhui',
'rpms',
'cron',
'adfs',
'icmp',
'wget',
'gbps',
'mbps',
'pcre',
'wtmp',
'isnt',
'inet',
'clis',
'acls',
'linx',
'cors',
'bdev',
'psql',
'clen',
'dism',
'ssms',
'rdns',
'ssrf',
'parm',
'finy',
'libl',
'manu',
'fqdn',
'fsck',
'rool',
'ntpd',
'vapt',
'byos',
'raju',
'sftp',
'ethx',
'eips',
'sadc',
'byol',
'math']

ec2_abbr = abbr_3 + abbr_4 + abbr_5

In [13]:
#text cleaning
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.tokenize import word_tokenize
from language_detector import detect_language
from nltk.stem import WordNetLemmatizer
import pkg_resources
from symspellpy import SymSpell, Verbosity

sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
    pass
else:
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    
lemmatizer = WordNetLemmatizer()

word_count = {}
    

def f_base(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    # normalization 1: xxxThis is a --> xxx. This is a (missing delimiter)
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', s)  # before lower case
    # normalization 2: lower case
    s = s.lower()
    # normalization 3: "&gt", "&lt"
    s = re.sub(r'&gt|&lt', ' ', s)
    # normalization 4: letter repetition (if more than 2)
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # normalization 5: non-word repetition (if more than 1)
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # normalization 6: string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # normalization 7: stuff in parenthesis, assumed to be less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # normalization 8: xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # normalization 9: [.?!] --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    # normalization 10: ' ing ', noise text
    s = re.sub(r' ing ', ' ', s)
    # normalization 11: noise text
    s = re.sub(r'product received for free[.| ]', ' ', s)
    # normalization 12: phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)

    return s.strip()


# language detection
def f_lan(s):
    """
    :param s: string to be processed
    :return: boolean (s is English)
    """

    # some reviews are actually english but biased toward french
    return detect_language(s) in {'English'}


###############################
#### word level preprocess ####
###############################

# filtering out punctuations and numbers
def f_punct(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with punct and number filter out
    """
    return [word for word in w_list if word.isalpha() or word in(ec2_abbr)]


# selecting nouns
def f_noun(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with only nouns selected
    """
    return [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN' or word in(ec2_abbr)]


# typo correction
def f_typo(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with typo fixed by symspell. words with no match up will be dropped
    """
    w_list_fixed = []
    for word in w_list:
        if word not in ec2_abbr:
            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
            if suggestions:
                w_list_fixed.append(suggestions[0].term)
            else:
                pass
        else:
            w_list_fixed.append(word)
                # do word segmentation, deprecated for inefficiency
                # w_seg = sym_spell.word_segmentation(phrase=word)
                # w_list_fixed.extend(w_seg.corrected_string.split())
    return w_list_fixed


# stemming if doing word-wise
p_stemmer = PorterStemmer()


def f_stem(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with stemming
    """
    return [p_stemmer.stem(word) if word not in ec2_abbr else word for word in w_list]

def f_lemm(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with lemmatizing
    """
    return [lemmatizer.lemmatize(word) if word not in ec2_abbr else word for word in w_list]


# filtering out stop words
# create English stop words list

stop_words = (list(
    set(get_stop_words('en'))
))

def f_stopw(w_list):
    """
    filtering out stop words
    """
    return [word for word in w_list if word not in stop_words]

# def f_freqw(w_list):
    
def word_counter(w_list):
    for w in w_list:
        if w in word_count.keys():
            word_count[w] += 1
        else:
            word_count[w] = 1
    return word_count


def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw review texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    s = f_base(rw)
    if not f_lan(s):
        return None
    return s

def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = f_punct(w_list)
    w_list = f_noun(w_list)
#     w_list = f_typo(w_list)
    w_list = f_lemm(w_list)
    w_list = f_stopw(w_list)
    word_count = word_counter(w_list)
    

    return w_list

def process_text(text):
    try:
        pp_sent = preprocess_sent(text)
        pp_word = preprocess_word(pp_sent)
        return pp_word
    except:
        return []


In [14]:
data['comm_body'].dropna(inplace=True)
data['processed_text'] = data['comm_body'].apply(lambda x: process_text(x))

In [15]:
from collections import Counter
top_most_common_words = dict(Counter(word_count).most_common(100)).keys()

In [16]:
word_to_remove = ['jan', 'feb', 'mar', 'march', 'april', 'may', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec',
'need', 'help', 'httpd', 'html']

In [20]:
data.dropna(inplace=True)
data['processed_text'].isnull().sum()

0

In [21]:
data['processed_text'] = data['processed_text'].apply(lambda x: ' '.join([w for w in x if w not in top_most_common_words and w not in word_to_remove]))

In [22]:
data = data[data['processed_text'] != ' ']
data = data.dropna()

In [24]:
print(data['comm_body'][2])
print(data['processed_text'][2])

Hi Raghavendra,   Just a quick recap of our chat today.   You reached out because commands were not executing on EC2 instance i-07e6b45549a66edd3. We were not able to complete our investigation as you became unresponsive on chat and, therefore, I had to disconnect. We ran the 'echo $SHELL' command to confirm that you were using a bash shell. The next thing I wanted to check was the path variable - could you please check this?  This is what a standard path variable looks like on an Amazon Linux 2 instance: ---------- $ echo $PATH /usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/ec2-user/.local/bin:/home/ec2-user/bin ----------  Please provide the output of the same command so we can compare.   We value your feedback. Please share your experience by rating this correspondence using the AWS Support Center link at the end of this correspondence. Each correspondence can also be rated by selecting the stars in top right corner of each correspondence within the AWS Support Center.  Bes

In [25]:
data.to_csv('data/ntm_processed_data.csv', index=False)